## ETHZ-01-03-01 Co-seismic deformation maps

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype.

### Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

### <a name="objective">Objective 




### <a name="data">Data

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

All data products are distributed in the SENTINEL Standard Archive Format for Europe (SAFE) format.

Data products are available in single polarisation (VV or HH) for Wave mode and dual polarisation (VV+VH or HH+HV) and single polarisation (HH or VV) for SM, IW and EW modes.

### <a name="service">Service definition

In [5]:
service = dict([('title', 'Co-seismic deformation maps'),
                ('abstract', 'Co-seismic deformation maps'),
                ('id', 'ewf-ethz-01-03-01')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of master and slave products' identifiers

In [2]:
input_identifiers = ['S1A_IW_SLC__1SDV_20161030T163141_20161030T163208_013722_01603F_4094', 'S1A_IW_SLC__1SDV_20161018T163141_20161018T163208_013547_015AEB_5994']

**Input references**

This is the Sentinel-1 stack catalogue references

In [3]:
input_references = ['https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161030T163141_20161030T163208_013722_01603F_4094', 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20161018T163141_20161018T163208_013547_015AEB_5994']

**Data path**

This path defines where the data is staged-in. 

In [1]:
data_path = "/workspace/data"

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
from snappy import jpy
from snappy import ProductIO

#### Read the products

In [2]:
s1meta = "manifest.safe"

products = []

for s1path in input_identifiers:

    s1prd= "%s/%s/%s.SAFE/%s" % (data_path, s1path, s1path, s1meta)
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    products.append(product)

## <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.